In [1]:
# buy and hold; 
# assume picking BTC, ETH and XRP
# weights: 0.5, 0.3, 0.2

import pandas as pd
import numpy as np
import yfinance as yf

# Define the portfolio assets and initial investment
tickers = ['BTC-USD', 'ETH-USD', 'XRP-USD']
initial_investment = 2000000

# Define initial portfolio weights: 50% BTC, 30% ETH, 20% XRP
allocations = np.array([0.5, 0.3, 0.2]) * initial_investment

# Define the time period
start_date = '2020-01-01'
end_date = '2024-12-31'

# Fetch historical data
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns
daily_returns = data.pct_change().dropna()

# Compute portfolio daily returns (flexible allocation)
weights = allocations / allocations.sum()
portfolio_daily_returns = daily_returns @ weights  #The @ operator in Python is the matrix multiplication (dot product) operator

# Compute total return
initial_prices = data.iloc[0]
final_prices = data.iloc[-1]

# Compute final portfolio value
final_value = (final_prices / initial_prices) * allocations
ending_balance = final_value.sum()

# Compute average daily return
avg_daily_return = portfolio_daily_returns.mean()

# Compute annualized standard deviation
annual_std_dev = portfolio_daily_returns.std() * np.sqrt(365)

# Compute Sharpe Ratio (assuming risk-free rate = 0)
sharpe_ratio = avg_daily_return * 365 / annual_std_dev

# Print results
print(f"Ending Balance: ${ending_balance:,.2f}")
print(f"Average Daily Return: {avg_daily_return:.6f}")
print(f"Annual Standard Deviation: {annual_std_dev:.6f}")
print(f"Annual Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  3 of 3 completed
Ending Balance: $32,534,627.71
Average Daily Return: 0.002379
Annual Standard Deviation: 0.693298
Annual Sharpe Ratio: 1.25


In [2]:
# Dynamic allocation based on Bitcoin RSI

import pandas as pd
import numpy as np
import yfinance as yf

# Get Bitcoin historical data from Yahoo Finance
btc_data = yf.download("BTC-USD", start="2020-01-01", end="2024-12-31")

# Calculate daily returns
btc_data['Return'] = btc_data['Adj Close'].pct_change()

# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    delta = data['Adj Close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window, min_periods=1).mean()
    avg_loss = pd.Series(loss).rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

btc_data['RSI'] = calculate_rsi(btc_data)

# Initialize portfolio
initial_investment = 2_000_000
cash = initial_investment
bitcoin = 0
portfolio_value = []  # initialize an empty list to store total_value

# portfolio over time
for i in range(1, len(btc_data)):
    rsi = btc_data['RSI'].iloc[i]
    btc_price = btc_data['Adj Close'].iloc[i]

    # Update portfolio value based on previous day's holdings
    cash *= 1 + (0.02 / 365)  # Daily cash interest
    bitcoin_value = bitcoin * btc_price
    total_value = cash + bitcoin_value
    portfolio_value.append(total_value) 
    # append works in the following way: portfolio_value = []; portfolio_value.append(100)  # Now portfolio_value = [100] ...

    # Adjust portfolio based on RSI
    if rsi > 70:
        # 50% Bitcoin, 50% cash
        target_bitcoin_value = total_value * 0.5
        target_cash_value = total_value * 0.5
    elif rsi < 30:
        # 100% Bitcoin
        target_bitcoin_value = total_value
        target_cash_value = 0
    else:
        # 100% cash
        target_bitcoin_value = 0
        target_cash_value = total_value

    # Execute trades to rebalance portfolio
    bitcoin = target_bitcoin_value / btc_price
    cash = target_cash_value

# Final portfolio value
final_value = portfolio_value[-1]
print(f"Final portfolio value on 12/31/2024: ${final_value:,.2f}")


[*********************100%***********************]  1 of 1 completed
Final portfolio value on 12/31/2024: $2,210,335.78


In [5]:
# sentiment-based investment

import pandas as pd
import yfinance as yf
import numpy as np

# Define the initial state
assets = ['BTC-USD']  # Only Bitcoin is used for this strategy
investment_amount = 2000000  # Total investment
cash_rate = 0.02  # 2% annual cash return

# Define the time period
start_date = '2020-01-01'
end_date = '2024-12-31'

# Download historical BTC data (only keep adjusted close price)
data = yf.download(assets, start=start_date, end=end_date)[['Adj Close']]

# Load monthly sentiment index
file_path = "fear_greed_index.csv"
df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])

# Extract year and month
df['month'] = df['date'].dt.to_period('M')

# Calculate the monthly average sentiment value
monthly_sentiment = df.groupby('month')['value'].mean().reset_index()
monthly_sentiment.rename(columns={'value': 'monthly_sentiment'}, inplace=True)

# Shift the sentiment index to use prior month's value
monthly_sentiment['prior_month'] = monthly_sentiment['month'].shift(-1)
monthly_sentiment_dict = monthly_sentiment.set_index('prior_month')['monthly_sentiment'].to_dict()

# Initialize portfolio
portfolio_balance = investment_amount
btc_units = 0
cash_balance = investment_amount
position = None  # None = cash, '50% in', '100% in'
portfolio_values = []

# Iterate through BTC price data
for date, row in data.iterrows():
    month = date.to_period('M')
    avg_sentiment = monthly_sentiment_dict.get(month, np.nan)  # Retrieve sentiment or NaN if missing
    price = row['Adj Close']

    if np.isnan(avg_sentiment):  # Skip if no sentiment data for the month
        continue

    if position is None:
        if avg_sentiment < 50:
            btc_units = portfolio_balance / price
            cash_balance = 0
            position = '100% in'
        elif avg_sentiment > 50:
            btc_units = (portfolio_balance / 2) / price
            cash_balance = portfolio_balance / 2
            position = '50% in'
    else:
        if position == '50% in' and avg_sentiment > 80:
            position = None  # Move to cash
            btc_units = 0
            cash_balance = portfolio_balance
        elif position == '100% in' and avg_sentiment < 20:
            position = None  # Move to cash
            btc_units = 0
            cash_balance = portfolio_balance
        else:
            cash_balance *= (1 + cash_rate / 365)  # Daily cash return

    # Compute new portfolio balance
    portfolio_balance = btc_units * price + cash_balance
    portfolio_values.append({'Date': date, 'Portfolio Value': portfolio_balance, 'BTC Units': btc_units, 'Cash Balance': cash_balance})

# Convert to DataFrame
portfolio_df = pd.DataFrame(portfolio_values)
portfolio_df.set_index('Date', inplace=True)

# Calculate daily returns
portfolio_df['Daily Return'] = portfolio_df['Portfolio Value'].pct_change()

# Print the ending portfolio balance
ending_balance = portfolio_df.iloc[-1]['Portfolio Value']
print(f"Ending Portfolio Balance: ${ending_balance:,.2f}")




[*********************100%***********************]  1 of 1 completed
Ending Portfolio Balance: $22,222,469.89
